In [ ]:
%matplotlib inline
import daft
import pymc
import matplotlib.pyplot as plt
import matplotlib as mlp
import numpy as np
import ggplot
import pandas as pd

COLORS = ["#348ABD", "#A60628", "#7A68A6"]

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
